In [1]:
import pyspark
from pyspark import SparkContext, SQLContext


sc = pyspark.SparkContext('local[*]')
sqlc = SQLContext(sc)

In [2]:
pyspark??

In [3]:

recipes = sqlc.read.json("data/recipes.json")


In [4]:
recipes.registerTempTable("recipes")
chili = sqlc.sql("select * from recipes where lower(ingredients) like '%chili%'")

In [5]:
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from pyspark.sql.functions import col, udf, lower
from pyspark.sql.types import IntegerType, BooleanType, ArrayType, StringType

In [6]:
recipes = recipes.withColumn("ingredients_lower", lower(col("ingredients")));


In [7]:
tokenizer = Tokenizer(inputCol="ingredients_lower", outputCol="tok_ingredients")
recipes = tokenizer.transform(recipes)


In [8]:
#recipes.withColumn("tokenized_ingredients")

regexTokenizer = RegexTokenizer(inputCol="ingredients_lower", outputCol="reg_tok_ingredients", pattern="\\W")

recipes = regexTokenizer.transform(recipes)



In [9]:
countTokens = udf(lambda words: len(words), ArrayType(StringType()))
#sqlc.udf.register("countTokens", lambda words: len(words))

In [10]:

recipes = recipes.withColumn("cnt_tok", countTokens(col("reg_tok_ingredients")))

In [ ]:
def has_chili(tok_ings):
    for tok in tok_ings:
        if (levenshtein(tok, "chili") <= 1 or levenshtein(tok, "chilies") <= 1):
            return True
    return False

has_chili_udf = udf(has_chili, BooleanType())
recipes = recipes.withColumn("has_chili", has_chili_udf(col("reg_tok_ingredients")))

In [11]:
recipes.take(5)

Py4JJavaError: An error occurred while calling z:org.apache.spark.sql.execution.EvaluatePython.takeAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 2, localhost): java.lang.IllegalArgumentException: port out of range:459092027
	at java.net.InetSocketAddress.checkPort(InetSocketAddress.java:143)
	at java.net.InetSocketAddress.<init>(InetSocketAddress.java:188)
	at java.net.Socket.<init>(Socket.java:244)
	at org.apache.spark.api.python.PythonWorkerFactory.createSocket$1(PythonWorkerFactory.scala:76)
	at org.apache.spark.api.python.PythonWorkerFactory.liftedTree1$1(PythonWorkerFactory.scala:91)
	at org.apache.spark.api.python.PythonWorkerFactory.createThroughDaemon(PythonWorkerFactory.scala:90)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:63)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:134)
	at org.apache.spark.a

In [ ]:
recipes.withColumn(columnName, lower(col(columnName)));
tokenizer = Tokenizer(inputCol="ingredients", outputCol="tok_ingredients")

tokenized = tokenizer.transform(sentenceDataFrame)


In [ ]:
recipes.withColumn("withIngredients")

In [ ]:
chili.toPandas()

In [ ]:
recipes.filter(lambda recipe: recipe['ingredients'].

In [ ]:
chili.toPandas()

In [ ]:
recipes.select('cookTime').take(100)

In [ ]:
chili.cook
    
    
